In [54]:
import pandas as pd

In [55]:
%run ../functions.ipynb

### Import data & model

In [56]:
#get sample dataset
#sub = pd.read_csv('../data/MDataFiles_Stage1/MSampleSubmissionStage1.csv') #first stage

sub = pd.read_csv('../data/MDataFiles_Stage2/MSampleSubmissionStage2.csv') #second stage
sub = process_sample_sub(sub)

In [57]:
sub = sub[['ID', 'Season', 'Team1', 'Team2']]

In [58]:
#processed data
all_games = pd.read_csv('../feature_engineering/output/all_games.csv')

In [59]:
#model
import pickle

with open("../model_build/model/logreg.pkl", "rb") as file: 
    logreg = pickle.load(file)
    


### Prediction

In [60]:
all_games = all_games.fillna(0)

In [61]:
pred_games = all_games[all_games.type == 1] # get only games not yet played

feature_list = ['seed_diff',  
                't1_adj_margin','t2_adj_margin',
                't1_final_rank', 't2_final_rank',
                't1_OrdinalRank', 't2_OrdinalRank']
 
                
pred_games['Pred'] = logreg.predict_proba(pred_games[feature_list])[:,1]

/Users/skylerdale/.virtualenvs/ncaa_model/lib/python3.8/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/var/folders/2g/465yxy_x4g786jx2llr5xqh40000gn/T/ipykernel_61236/93386179.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_games['Pred'] = logreg.predict_proba(pred_games[feature_list])[:,1]


In [62]:
all_games[['ID', 'Pred']].to_csv('submission.csv', index = False)

### Join with Team Names for exploration

In [63]:
data_path = '../data/MDataFiles_Stage2/' #this is for the second stage of data
teams = pd.read_csv(data_path + 'MTeams.csv')


In [50]:
pred_games = pred_games.merge(teams[['TeamID', 'TeamName']], left_on = 'Team1', right_on = 'TeamID')
pred_games = pred_games.rename(columns = {'TeamName': 'Team1_name'})
pred_games = pred_games.merge(teams[['TeamID', 'TeamName']], left_on = 'Team2', right_on = 'TeamID')
pred_games = pred_games.rename(columns = {'TeamName': 'Team2_name'})

In [51]:
pred_games.Team1_name = pred_games.Team1_name.str.lower()
pred_games.Team2_name = pred_games.Team2_name.str.lower()

In [52]:
fourteen_three = pred_games[((pred_games.t1_Seed == 3) & 
                           (pred_games.t2_Seed == 14) &
                           (pred_games.Season == 2021) 
                          )][['ID', 'Team1_name', 'Team2_name', 'Pred']]

In [53]:
fourteen_three

,ID,Team1_name,Team2_name,Pred
31,2021_1116_1159,arkansas,colgate,0.464988
94,2021_1116_1186,arkansas,e washington,0.821300
564,2021_1116_1287,arkansas,morehead st,0.845987
587,2021_1242_1287,kansas,morehead st,0.854904


In [339]:
fourteen_three['Pred2'] = 1 - fourteen_three['Pred']

In [333]:
fourteen_three['Pred'].mean()

0.1650013922867077

In [340]:
fourteen_three['Pred2'].mean()

0.16501969892436016

In [407]:
game_lookup(all_games, 'colorado', 'michigan', season = 2021)

,ID,Team1_name,Team2_name,Pred
474,2021_1160_1276,colorado,michigan,0.353046
505,2021_1160_1277,colorado,michigan st,0.746340


In [304]:
game_lookup(all_games, 'florida st', 'vermont', season = 2019)

,ID,Team1_name,Team2_name,Pred
3408,2019_1199_1436,florida st,vermont,0.814959


In [305]:
game_lookup(all_games, 'Kansas St', 'Uc irvine', season = 2019)

,ID,Team1_name,Team2_name,Pred
4373,2019_1243_1414,kansas st,uc irvine,0.780656


In [306]:
game_lookup(all_games, 'Kansas', 'northeastern', season = 2019)

,ID,Team1_name,Team2_name,Pred
4322,2019_1242_1318,kansas,northeastern,0.815838


In [309]:
game_lookup(all_games, 'Arizona', 'buffalo', season = 2018)

,ID,Team1_name,Team2_name,Pred
3647,2018_1112_1138,arizona,buffalo,0.76695
3648,2019_1113_1138,arizona st,buffalo,0.28204


In [318]:
game_lookup(all_games, 'gonzaga', 'u', season = 2018)

,ID,Team1_name,Team2_name,Pred
217,2006_1211_1417,gonzaga,ucla,0.433957
218,2015_1211_1417,gonzaga,ucla,0.838355
219,2021_1211_1417,gonzaga,ucla,0.911394
2296,2021_1211_1364,gonzaga,uc santa barbara,0.938073
2689,2018_1211_1422,gonzaga,unc greensboro,0.862989
2690,2021_1211_1422,gonzaga,unc greensboro,0.961901
2747,2021_1211_1425,gonzaga,usc,0.822869
2817,2021_1211_1429,gonzaga,utah st,0.919718
3450,2016_1211_1428,gonzaga,utah,0.575548


In [ ]:
game_lookup(all_games, 'gonzaga', 'u', season = 2018)

In [ ]:
# 23, 12, 15, 14 -- 2018
# 13, 19, 22, 19 -- 2019
#19, 21, 28, 25 -- 2021

In [319]:
(23 + 12 + 15 + 14) / 4

16.0

In [307]:
(13 + 19 + 22 + 19) / 4 

18.25

In [308]:
(19 + 21 + 28 + 25) / 4

23.25

In [299]:
def game_lookup(df, t1, t2, features = ['ID', 'Team1_name', 'Team2_name', 'Pred'], season = 2021):

    t1 = t1.lower()
    t2 = t2.lower()
    
    if t1 < t2:
        return df[ (df.Team1_name.str.startswith(t1)) & 
                         (df.Team2_name.str.startswith(t2)) ][features]
    else:
        return df[ (df.Team1_name.str.startswith(t2)) & 
                         (df.Team2_name.str.startswith(t1)) ][features]
 

In [342]:
#15 - 2

pd.concat([game_lookup(all_games, 'ohio st', 'oral'), 
           game_lookup(all_games, 'hou', 'clev'),
          game_lookup(all_games, 'alabama', 'iona'),
          game_lookup(all_games, 'iowa', 'grand')
          ])

,ID,Team1_name,Team2_name,Pred
1971,2021_1326_1331,ohio st,oral roberts,0.907590
906,2021_1156_1222,cleveland st,houston,0.043788
958,2021_1104_1233,alabama,iona,0.921044
1021,2021_1213_1234,grand canyon,iowa,0.091959


In [341]:
#14 - 3

pd.concat([game_lookup(all_games, 'we', 'mo'), 
           game_lookup(all_games, 'e w', 'ka'),
          game_lookup(all_games, 'ark', 'colg'),
          game_lookup(all_games, 'texas', 'abile')
          ])

,ID,Team1_name,Team2_name,Pred
3113,2021_1287_1452,morehead st,west virginia,0.176077
3114,2010_1288_1452,morgan st,west virginia,0.061838
1057,2021_1186_1242,e washington,kansas,0.168621
567,2021_1116_1159,arkansas,colgate,0.464988
2544,2021_1101_1400,abilene chr,texas,0.313014
2615,2021_1101_1403,abilene chr,texas tech,0.323417


In [ ]:
# 17, 17, 31, 54

In [270]:
#13 - 4

pd.concat([game_lookup(all_games, 'virg', 'ohio'), 
           game_lookup(all_games, 'unc g', 'florida s'),
          game_lookup(all_games, 'north tex', 'pur'),
          game_lookup(all_games, 'oklahoma s', 'lib')
          ])

,ID,Team1_name,Team2_name,Pred
1930,2021_1325_1438,ohio,virginia,0.194837
1931,2021_1326_1438,ohio st,virginia,0.565400
1992,2021_1325_1439,ohio,virginia tech,0.318271
1993,2021_1326_1439,ohio st,virginia tech,0.715095
1612,2021_1199_1422,florida st,unc greensboro,0.797556
1073,2021_1317_1345,north texas,purdue,0.283391
888,2021_1251_1329,liberty,oklahoma st,0.250539


In [260]:
#12 - 5
pd.concat([game_lookup(all_games, 'uc santa', 'cre'), 
           game_lookup(all_games, 'color', 'georgeto'),
          game_lookup(all_games, 'tenn', 'oregon st'),
          game_lookup(all_games, 'villa', 'wint')])

,ID,Team1_name,Team2_name,Pred
1187,2021_1166_1364,creighton,uc santa barbara,0.737243
145,2021_1160_1207,colorado,georgetown,0.763195
1371,2021_1333_1397,oregon st,tennessee,0.221124
2206,2021_1437_1457,villanova,winthrop,0.794843


In [266]:
#11 - 6

pd.concat([game_lookup(all_games, 'usc', 'drake'), 
           game_lookup(all_games, 'usc', 'wich'),
          game_lookup(all_games, 'syra', 'san die'),
          game_lookup(all_games, 'texas t', 'utah s'),
           game_lookup(all_games, 'byu', 'michigan s'),
            game_lookup(all_games, 'byu', 'ucla'),
          ])

,ID,Team1_name,Team2_name,Pred
1665,2021_1179_1425,drake,usc,0.226869
2138,2021_1425_1455,usc,wichita st,0.818239
1323,2021_1361_1393,san diego st,syracuse,0.711787
1765,2021_1403_1429,texas tech,utah st,0.608665
501,2021_1140_1277,byu,michigan st,0.724025
1545,2021_1140_1417,byu,ucla,0.628415


In [263]:
#10 - 7

pd.concat([game_lookup(all_games, 'orego', 'vcu'), 
           game_lookup(all_games, 'conn', 'mary'),
          game_lookup(all_games, 'clem', 'rutg'),
          game_lookup(all_games, 'florida', 'virginia t')])

,ID,Team1_name,Team2_name,Pred
1814,2021_1332_1433,oregon,vcu,0.588257
1815,2021_1333_1433,oregon st,vcu,0.351475
445,2021_1163_1268,connecticut,maryland,0.605717
1087,2021_1155_1353,clemson,rutgers,0.500835
1968,2021_1196_1439,florida,virginia tech,0.515983
1969,2021_1199_1439,florida st,virginia tech,0.651138


In [264]:
# 9 - 8
pd.concat([game_lookup(all_games, 'oklah', 'missour'), 
           game_lookup(all_games, 'lsu', 'st b'),
          game_lookup(all_games, 'north c', 'wiscon'),
          game_lookup(all_games, 'loyola', 'georgia t')])

,ID,Team1_name,Team2_name,Pred
853,2021_1281_1328,missouri,oklahoma,0.493813
894,2021_1281_1329,missouri,oklahoma st,0.409455
1254,2021_1261_1382,lsu,st bonaventure,0.492229
2248,2021_1314_1458,north carolina,wisconsin,0.433576
396,2021_1210_1260,georgia tech,loyola-chicago,0.409407


In [412]:
game_lookup(all_games, 'syracuse', 'west')

,ID,Team1_name,Team2_name,Pred
2067,2021_1393_1452,syracuse,west virginia,0.314791


,ID,Team1_name,Team2_name,Pred
1378,2021_1101_1400,abilene chr,texas,0.313014
1431,2021_1101_1403,abilene chr,texas tech,0.323417


In [78]:
all_games[ (all_games.Team1_name.str.contains('Georg')) & (all_games.Team2_name.str.contains('Loyola')) ]

,Unnamed: 0,type,ID,Pred,Season,Team1,Team2,Outcome,avg_rank_x,t1_final_rank,...,t1_adj_oe_0,t1_adj_de_0,t2_adj_oe_0,t2_adj_de_0,t1_adj_margin,t2_adj_margin,TeamID_x,Team1_name,TeamID_y,Team2_name
395,3243,1,2021_1207_1260,0.284340,2021,1207,1260,0.0,51,81.876843,...,104.980543,98.533603,113.798182,89.56814,6.446940,24.230042,1207,Georgetown,1260,Loyola-Chicago
396,3292,1,2021_1210_1260,0.409407,2021,1210,1260,0.0,36,83.919965,...,116.173329,100.111888,113.798182,89.56814,16.061441,24.230042,1210,Georgia Tech,1260,Loyola-Chicago


In [56]:
teams[teams['TeamName'].str.contains('Georg')]

,TeamID,TeamName,FirstD1Season,LastD1Season
105,1206,George Mason,1985,2021
106,1207,Georgetown,1985,2021
107,1208,Georgia,1985,2021
108,1209,Georgia St,1985,2021
109,1210,Georgia Tech,1985,2021


In [15]:

teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2021
1,1102,Air Force,1985,2021
2,1103,Akron,1985,2021
3,1104,Alabama,1985,2021
4,1105,Alabama A&M,2000,2021
